In [1]:
import os

In [6]:
import pandas as pd
import numpy as np

In [7]:
l=[1,2,3,4,np.nan,5,6,"Hello"]

In [16]:
l=pd.Series(l)

In [17]:
l

0        1
1        2
2        3
3        4
4      NaN
5        5
6        6
7    Hello
dtype: object

In [23]:
l=pd.to_numeric(l,errors='coerce')

In [24]:
a=pd.DataFrame(l)

In [25]:
a.rename(columns={0:'Data'})

,Data
0,1.0
1,2.0
2,3.0
3,4.0
4,NaN
5,5.0
6,6.0
7,NaN


In [26]:
a.fillna(a.mean(),inplace=True)

In [27]:
import plotly.express as px

,0
0,1.0
1,2.0
2,3.0
3,4.0
4,3.5
5,5.0
6,6.0
7,3.5


In [2]:
import os

In [3]:
os.chdir('E:\Data Sets')

In [6]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os

In [11]:
df=pd.read_csv('Outlier.csv')

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import re
temp='\\temp.csv'
path=os.getcwd()
path=path+temp
temp1='\\temp1.csv'
path1=os.getcwd()
path1=path1+temp
temp2='\\temp2.csv'
path2=os.getcwd()
path2=path2+temp

def upload_csv(zxc):
    if zxc:
        df=pd.read_csv(zxc)
        st.dataframe(df)
        return df
    

def upload_xlsx(zxc):
    if zxc:
        df=pd.read_excel(zxc)
        st.dataframe(df)
        return df

def mvt_mean(df):
    if len(df.select_dtypes(include='object').columns)!=0:
        new_df=df.fillna(df.mean())
        new_df=new_df.fillna(df.select_dtypes(include='object').mode().iloc[0])
        st.dataframe(new_df)
        new_df.to_csv(path,index=False)
        return new_df
    else:
        new_df=df.fillna(df.mean())
        st.dataframe(new_df)
        new_df.to_csv(path,index=False)
        return new_df

def mvt_median(df):
    if len(df.select_dtypes(include='object').columns)!=0:
        new_df=df.fillna(df.median())
        new_df=new_df.fillna(df.select_dtypes(include='object').mode().iloc[0])
        st.dataframe(new_df)
        new_df.to_csv(path,index=False)
        return new_df
    else:
        new_df=df.fillna(df.median())
        st.dataframe(new_df)
        new_df.to_csv(path,index=False)
        return new_df

def mvt_mode(df):
    if len(df.select_dtypes(include='number').columns)==0:
        new_df=df.fillna(df.select_dtypes(include='object').mode().iloc[0])
        st.dataframe(new_df)
        new_df.to_csv(path,index=False)
        return new_df
    else:
        st.write('You cannot impute using Mode as the dataset contains only numerical variables')

def export_data():
    with open('temp.csv','rb') as f:
        #contents=csv.reader(f)
        df=pd.read_csv(path)
        st.sidebar.download_button(label='Download CSV',data=f,mime='text/csv',file_name='Download.csv')
        return
    
def export_ml_data():
    with open('temp2.csv','rb') as f:
        #contents=csv.reader(f)
        df=pd.read_csv(path2)
        st.sidebar.download_button(label='Download CSV',data=f,mime='text/csv',file_name='Download.csv')
        return
def input_split(x):
    l=[]
    for i in re.split("[^0-9.]", x):
        if i != "":
            l.append(float(i))
    return l
def Logistic_Regression():
    parameters={'penalty':[],'solver':[]}
    with st.form(key='my-form'):
        penalty=st.selectbox('Penalty',('l1','l2'))
        if penalty:
            parameters['penalty'].append(penalty)
        solver=st.selectbox('Select LIBLINEAR for L1 or LBFS for L2',('liblinear','lbfgs'))
        if solver:
            parameters['solver'].append(solver)
        submit_button = st.form_submit_button(label='Submit') 
    df=pd.read_csv(path)
    cat_col=df.select_dtypes(include='object').columns
    for i in cat_col:
        df[i] = df[i].astype('category').cat.codes
    st.dataframe(df)
    x = df.iloc[:, :-1].values
    y = df.iloc[:,-1].values  
    from sklearn.preprocessing import StandardScaler 
    std=StandardScaler()
    x=std.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)
    from sklearn.linear_model import LogisticRegression
    logmodel = LogisticRegression(max_iter=500)
    logmodel.fit(x_train,y_train)
    from sklearn.model_selection import cross_val_score
    #lambdaval = [0.75,0.78,0.79,0.80,0.81,0.82,0.83,0.84]
    #lambdaval=[]
    if st.checkbox('Hyperparameters'):
        numbers = st.text_input("Please enter hyperparameters float types numbers separated by comma")
        if numbers:
            lambdaval=[float(i) for i in re.split('[^0-9.]', numbers) if i != ""]
            st.write(list(lambdaval))
            st.write(len(lambdaval))
            cv_scores=[]
            for i,v in enumerate(lambdaval):
                logmodel = LogisticRegression(C=lambdaval[i])
                scores = cross_val_score(logmodel,x_train,y_train,cv=5, scoring='accuracy')
                cv_scores.append(scores.mean())
            MSE = [1-x for x in cv_scores]
            optimal_l = lambdaval[MSE.index(min(MSE))]
            st.write('The Optimal Value of LAMBDA(Hyperparameter) is',optimal_l)
            t1=pd.DataFrame(lambdaval,columns=['lambdaval'])
            t2=pd.DataFrame(MSE,columns=['MSE'])
            chart_data=pd.concat([t1,t2],axis=1)
            fig=px.line(data_frame=chart_data,x='lambdaval',y='MSE')
            st.write(fig)
            st.write('Tuning the model with Optimal Hyperparameters')
            a=parameters['penalty'][0]
            b=parameters['solver'][0]
            log_optimal = LogisticRegression(penalty=a,C=optimal_l,tol=0.001,max_iter=10000,solver=b)            
            log_optimal.fit(x_train,y_train)
            y_pred_logit = log_optimal.predict(x_test)
            st.dataframe(y_pred_logit)
            #target_names = ['Positve(1)', 'Negative(0)']
            clsf_report = pd.DataFrame(classification_report(y_true=y_test,y_pred=y_pred_logit,output_dict=True)).transpose()
            cls=pd.DataFrame(clsf_report)
            st.dataframe(cls)
            #Pkl_Filename="Model.pkl"
            #with open(Pkl_Filename,'wb') as file:
                #pickle.dump(log_optimal,file)
            return
def get_parameters():
    parameters={'C':[],'gamma':[],'degree':[],'kernel':[]}
    with st.form(key='my-form'):
        numbers=st.text_input('Enter the hyperparameters for C in float format seprated by comma')
        if numbers:
            l1=[float(i) for i in re.split('[^0-9.]', numbers) if i != ""]
            for i,v in enumerate(l1):
                parameters['C'].append(l1[i])
        op=st.text_input('Enter the hyperparameters for Gamma in float format seprated by comma')
        if op:
            l2=[float(i) for i in re.split('[^0-9.]', op) if i != ""]
            for i,v in enumerate(l2):
                parameters['gamma'].append(l2[i])
        l3 = st.multiselect('Degree of Polynomial',(2,3,4,5))
        if l3:
            for i,v in enumerate(l3):
                parameters['degree'].append(l3[i])
        l4 = st.multiselect('Kernel',('rbf','poly','sigmoid'))
        if l4:
            for i,v in enumerate(l4):
                parameters['kernel'].append(l4[i])
        #st.form_submit_button()
        submit_button = st.form_submit_button(label='Submit')
    return parameters
        
    
    
def Support_Vector_Machine():
    parameters={'C':[],'gamma':[],'degree':[],'kernel':[]}
    with st.form(key='my-form'):
        #parameters={'C':[],'gamma':[],'degree':[],'kernel':[]}
        numbers=st.text_input('Enter the hyperparameters for C in float format seprated by comma')
        if numbers:
            l1=[float(i) for i in re.split('[^0-9.]', numbers) if i != ""]
            for i,v in enumerate(l1):
                parameters['C'].append(l1[i])
        op=st.text_input('Enter the hyperparameters for Gamma in float format seprated by comma')
        if op:
            l2=[float(i) for i in re.split('[^0-9.]', op) if i != ""]
            for i,v in enumerate(l2):
                parameters['gamma'].append(l2[i])
        l3 = st.multiselect('Degree of Polynomial',(2,3,4,5))
        if l3:
            for i,v in enumerate(l3):
                parameters['degree'].append(l3[i])
        l4 = st.multiselect('Kernel',('rbf','poly','sigmoid'))
        if l4:
            for i,v in enumerate(l4):
                parameters['kernel'].append(l4[i])
        #st.form_submit_button()
        submit_button = st.form_submit_button(label='Submit')
    if st.button('Run'):
        #s=0
        #for i,v in parameters.items():
        #s+=len(v)
        #if s>=3:
        df=pd.read_csv(path)
        cat_col=df.select_dtypes(include='object').columns
        for i in cat_col:
            df[i] = df[i].astype('category').cat.codes
        st.dataframe(df)
        x = df.iloc[:, :-1].values
        y = df.iloc[:,-1].values  
        std=StandardScaler()
        x=std.fit_transform(x)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)
        model_svm=SVC(probability=True)
        model_svm.fit(x_train,y_train)
        #p=get_parameters()
        grid_search=GridSearchCV(estimator=model_svm,param_grid=parameters,scoring='accuracy',cv=10,n_jobs=-1)
        grid_search.fit(x_train,y_train)
        st.write('Hyperparameters tuned')
        st.write(grid_search.best_params_)
        p=grid_search.best_params_
        model_svm_optimal=SVC(C=p['C'],kernel=p['kernel'],gamma=p['gamma'],degree=p['degree'])
        model_svm_optimal.fit(x_train,y_train)
        y_pred_test=model_svm_optimal.predict(x_test)
        y_pred_test=pd.DataFrame(y_pred_test,columns=['Y_Pred'])
        st.dataframe(y_pred_test)
        clsf_report = pd.DataFrame(classification_report(y_true=y_test,y_pred=y_pred_test,output_dict=True)).transpose()
        #d=classification_report(y_true=y_test,y_pred=y_pred_test,output_dict=True)
        cls=pd.DataFrame(clsf_report)
        st.dataframe(cls)
        return
    #else:
    #st.write('Cannot run until the hyperparameters are entered')
def k_means():
    df=pd.read_csv(path)
    st.write(df)
    num_columns=st.multiselect("Select two numerical features",((df.select_dtypes(include='number').columns)))
    inp=st.number_input('Enter the number of clusters')
    l=[]
    col=[]
    for i in range(int(inp)):
        l.append(i)
    if st.checkbox('Run KMeans Algo'):
        x=df.loc[:,num_columns].values
        st.write(x)
        std=StandardScaler()
        x=std.fit_transform(x)
        wcss=[]
        for i in range(1,11):
            kmeans=KMeans(n_clusters=i,init='k-means++')
            kmeans.fit(x)
            wcss.append(kmeans.inertia_)
        fig,ax=plt.subplots()    
        ax=plt.plot(range(1,11),wcss)
        plt.title('The Elbow Method')
        plt.xlabel('no of clusters')
        plt.ylabel('wcss')
        st.pyplot(fig)
        optimal_clusters=st.number_input('From the figure enter the optimal number of cluster')
        if st.checkbox('Tune'):
            kmeans=KMeans(n_clusters=int(optimal_clusters),init='k-means++')
            y_kmeans=kmeans.fit_predict(x)
            output=pd.concat([df,pd.DataFrame(y_kmeans)],axis=1)
            output.rename(columns={0:'Clusters'},inplace=True)
            fig,ax=plt.subplots()
            for i in range(int(inp)):
                ax.scatter(x[y_kmeans==l[i],0],x[y_kmeans==l[i],1],s=50, label='Cluster %i'%i)
                ax.legend()
            ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 20, c = 'black', label = 'Centroids')
            ax.legend()
            #plt.title('Clusters of customers')
            plt.xlabel(num_columns[0])
            plt.ylabel(num_columns[1])
            st.pyplot(fig)
            with st.form('my-form1'):
                l={}
                k=[i for i in range(int(optimal_clusters))]
                for i in k:
                    l[int(i)]=0
                st.write(l)    
                for i in range(int(optimal_clusters)):
                    x=st.text_input('Enter the name for Cluster %i wrt to the scatter plot'%i)
                    l[i]=x
                d = {int(k):v for k,v in l.items()}
                submit_button = st.form_submit_button(label='Submit')
            if st.button('Map the label names'):
                st.write(d)
                output.Clusters=output.Clusters.map(d)    
                st.dataframe(output)
                output.to_csv(path,index=False)
                export_data()
            
            #plt.xlabel('Annual Income (k$)')
            #plt.ylabel('Spending Score (1-100)')
            #st.pyplot(fig)
def DBScanClustering():
    df=pd.read_csv(path)
    st.write(df)
    num_columns=st.multiselect("Select two numerical features",((df.select_dtypes(include='number').columns)))
    if st.button('Run DBScan Algo'):
        x=df.loc[:,num_columns].values
        st.write(x)
        std=StandardScaler()
        x=std.fit_transform(x)
        dbscan=DBSCAN(eps=0.5,min_samples=4,metric='euclidean')
        y_pred=dbscan.fit(x)
        output=pd.concat([df,pd.DataFrame(y_pred.labels_)],axis=1)
        output.rename(columns={0:'Clusters'},inplace=True)
        fig=sns.lmplot(data=output, x=num_columns[0],y=num_columns[1],hue='Clusters',fit_reg=False)
        st.pyplot(fig)
        
st.title('Front end DPre/DV/ML Project ')
option = st.sidebar.selectbox('DSA Application',('Data-PreProcessing', 'Data-Visualization', 'Machine-Learning'))
st.sidebar.write('You selected:', option)
if option=='Data-PreProcessing':
    format_options=['csv','xlsx']
    a=st.sidebar.radio('Choose the file type',format_options)
    if a=='csv':
        b=st.sidebar.file_uploader("Choose a file",type='csv')
        if b :
            if st.sidebar.checkbox('Upload File'):
                df=upload_csv(b)
                opt=st.sidebar.radio('Impute Missing Value using',('Mean','Median','Mode'))
                st.sidebar.write('You selected',opt)
                if opt=='Mean':
                    mvt_mean(df)
                    export_data()
                elif opt=='Median':
                    mvt_median(df)
                    export_data()
                elif opt=='Mode':
                    mvt_mode(df)
                    export_data()
    elif a=='xlsx':
        b=st.sidebar.file_uploader("Choose a file",type='xlsx')
        if b :
            if st.sidebar.checkbox('Upload File'):
                df=upload_xlsx(b)
                opt=st.sidebar.radio('Impute Missing Value using',('Mean','Median','Mode'))
                st.sidebar.write('You selected',opt)
                if opt=='Mean':
                    mvt_mean(df)
                    export_data()
                elif opt=='Median':
                    mvt_median(df)
                    export_data()
                elif opt=='Mode':
                    mvt_mode(df)
                    export_data()
elif option=='Data-Visualization':
    format_options=['csv','xlsx']
    a=st.sidebar.radio('Choose the file type',format_options)
    if a=='csv':
        b=st.sidebar.file_uploader("Choose a file",type='csv')
        if b :
            if st.sidebar.checkbox('Upload File'):
                df=upload_csv(b)
                cat_columns=st.multiselect("Select categorical features",((df.select_dtypes(include='object').columns)))
                num_columns=st.multiselect("Select numerical features",((df.select_dtypes(include='number').columns)))
                if (len(cat_columns)==1 and len(num_columns)==1):
                    options=st.selectbox("Select Chart Type",['Bar-Chart','PieChart','KDEPlot'])
                    if options=='Bar-Chart':
                        st.bar_chart(df.groupby(cat_columns)[num_columns].mean())
                        #st.write(df[cat_columns].columns)
                    elif options=='PieChart':
                        x=df.groupby(cat_columns)[num_columns].sum()
                        d=pd.DataFrame(x).reset_index()
                        d=dict(d.values)
                        fig, ax1 = plt.subplots()
                        ax1.pie(list(d.values()),labels=list(d.keys()),autopct="%.1f%%")
                        st.pyplot(fig)
                    elif options=='KDEPlot':
                        cat=cat_columns[0]
                        num=num_columns[0]
                        #st.write(cat_columns)
                        fig=sns.FacetGrid(df,col=cat).map(sns.kdeplot,num)
                        st.pyplot(fig)
                elif (len(cat_columns)==1 and len(num_columns)==2):
                    options=st.selectbox("Select Chart Type",['Pair-Plot','ScatterPlot'])
                    if options=='Pair-Plot':
                        #st.write(num_columns[0])
                        #st.write(num_columns[1])
                        fig=sns.pairplot(df,hue=cat_columns[0], vars=[num_columns[0],num_columns[1]])
                        st.pyplot(fig)
                    elif options=='ScatterPlot': 
                        a=sns.scatterplot(data=df,x=num_columns[0],y=num_columns[1],hue=cat_columns[0])
                        st.pyplot(a.figure)
                elif (len(cat_columns)==0 and len(num_columns)==2):
                    options=st.selectbox("Select Chart Type",['Pair-Plot','ScatterPlot'])
                    if options=='Pair-Plot':
                        fig=sns.pairplot(df,vars=[num_columns[0],num_columns[1]])
                        st.pyplot(fig)
                    elif options=='ScatterPlot':
                        a=sns.scatterplot(data=df,x=num_columns[0],y=num_columns[1])
                        st.pyplot(a.figure)
elif option=='Machine-Learning':
    format_options=['csv','xlsx']
    a=st.sidebar.radio('Choose the file type',format_options)
    if a=='csv':
        b=st.sidebar.file_uploader("Choose a file",type='csv')
        if b :
            if st.sidebar.checkbox('Upload File'):
                df=upload_csv(b)
                opt=st.sidebar.radio('Impute Missing Value using',('Mean','Median','Mode'))
                st.sidebar.write('You selected',opt)
                if opt=='Mean':
                    mvt_mean(df)
                    
                elif opt=='Median':
                    mvt_median(df)
                    
                elif opt=='Mode':
                    mvt_mode(df)
                    
    elif a=='xlsx':
        b=st.sidebar.file_uploader("Choose a file",type='xlsx')
        if b :
            if st.sidebar.checkbox('Upload File'):
                df=upload_xlsx(b)
                opt=st.sidebar.radio('Impute Missing Value using',('Mean','Median','Mode'))
                st.sidebar.write('You selected',opt)
                if opt=='Mean':
                    mvt_mean(df)
                    
                elif opt=='Median':
                    mvt_median(df)
                
                elif opt=='Mode':
                    mvt_mode(df)

    #if st.sidebar.checkbox("Build"):
        #ml_options=['Linear Regression','Classification']
    st.sidebar.write('Choose a ML Method depending upon the data')
    if  st.sidebar.checkbox('Regression'):
        df=pd.read_csv(path)
        df1=pd.read_csv(path1)
        col_name = st.selectbox("Please Enter the name of column to predict",df.select_dtypes(exclude='object').columns)
        #y = df.loc[:,col_name].values
        #df=df.drop([col_name], axis = 1)
        #x = df.loc[:].values
        y=df[col_name]
        y1=df[col_name]
        df=df.drop([col_name], axis = 1)
        df1=df1.drop([col_name],axis=1)
        x=df
        st.dataframe(y)
        st.dataframe(x)
        from sklearn.preprocessing import LabelEncoder, OneHotEncoder
        from sklearn.compose import ColumnTransformer
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LinearRegression
        labelencoder = LabelEncoder()
        cat_columns=list(df.select_dtypes(include='object').columns)
        cat_columns1=list(df1.select_dtypes(include='object').columns)
        for i in range(len(cat_columns)):
            x.loc[:,cat_columns[i]] = labelencoder.fit_transform(x.loc[:,cat_columns[i]])
        st.dataframe(x)
        for i in range(len(cat_columns1)):
            df1.loc[:,cat_columns1[i]] = labelencoder.fit_transform(df1.loc[:,cat_columns1[i]])
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)
        regressor=LinearRegression()
        regressor.fit(x_train,y_train)
        st.write("Model Build Successfully")
        st.write("Please play with the sliders to give input")
        l=[]
        for i in df1:
            l.append(st.sidebar.slider('{}'.format(i),min(df1[i]),max(df1[i]),min(df1[i])))
        #st.sidebar.slider("Choose a range",0,100,60) 
        if st.button("Predict"):
            y_pred=regressor.predict([l])
            st.write("Your predicted {} is {:.2f}".format(col_name,float(y_pred)))
            y_pred1=regressor.predict(x_test)
            #st.dataframe(y_pred1)
            y_pred1=pd.DataFrame(y_pred1)
            y_pred1=y_pred1.rename(columns={0:'Predicted %s'%col_name})
            st.dataframe(y_pred1)
            from sklearn.metrics import r2_score
            coefficient_of_dermination = r2_score(y_test,y_pred1)
            st.write('R Square ',coefficient_of_dermination)
    elif st.sidebar.checkbox('Classification'):
        algo=['Logistic Regression','Support-Vector-Machine']
        ch = st.selectbox("Please Select the Classification Algorithm",algo)
        if ch=='Logistic Regression':
            Logistic_Regression()
        elif ch=='Support-Vector-Machine':
            Support_Vector_Machine()
    elif st.sidebar.checkbox('Clustering'):
        algo=['KMeans','DBScanClustering']
        ch = st.selectbox("Please Select the Clustering Algorithm",algo)
        if ch=='KMeans':
            k_means()
        elif ch=='DBScanClustering':
            DBScanClustering()            

In [ ]:
import pickle
Pkl_Filename="Model.pkl"
with open(Pkl_Filename,'wb') as file:
    pickle.dump(log_optimal,file)

In [ ]:
with open(Pkl_Filename,'rb') as file:
    logistic_model=pickle.load(file)

In [ ]:
import streamlit as st
import pickle
import os
import pandas as pd
os.getcwd()
os.chdir(r'C:\Users\win10\OneDrive\Desktop\Streamlit')
# "/app/{repository name}/ {file.extension}"
#Pkl_Filename = "/app/pickle_demo_streamlit/Model.pkl"
def upload_csv(zxc):
    if zxc:
        df=pd.read_csv(zxc)
        st.dataframe(df)
        return df
    
Pkl_Filename="Model.pkl"
with open(Pkl_Filename, 'rb') as file: 
    LR_model = pickle.load(file)
    

st.title('Logistic Regeression Model')
st.write('Upload the file containing Xi\'s')
b=st.file_uploader("Choose a file",type='csv')
if b:
    if st.checkbox('Upload File'):
        df=upload_csv(b)
        from sklearn.preprocessing import LabelEncoder, OneHotEncoder
        cat_columns=list(df.select_dtypes(include='object').columns)
        labelencoder = LabelEncoder()
        for i in range(len(cat_columns)):
                df.loc[:,cat_columns[i]] = labelencoder.fit_transform(df.loc[:,cat_columns[i]])
        x=df.iloc[:].values
        from sklearn.preprocessing import StandardScaler 
        std=StandardScaler()
        x=std.fit_transform(x)
        if st.button('Predict'):
            y_pred = LR_model.predict(x)
            st.dataframe(y_pred)

In [4]:
os.chdir(r'C:\Users\win10\OneDrive\Desktop\Streamlit')

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import re
parameters={'C':[],'gamma':[],'degree':[],'kernel':[]}
with st.form(key='my-form'):
    numbers=st.text_input('Enter the hyperparameters for C in float format seprated by comma')
    if numbers:
        l1=[float(i) for i in re.split('[^0-9.]', numbers) if i != ""]
        for i,v in enumerate(l1):
            parameters['C'].append(l1[i])
    op=st.text_input('Enter the hyperparameters for Gamma in float format seprated by comma')
    if op:
        l2=[float(i) for i in re.split('[^0-9.]', op) if i != ""]
        for i,v in enumerate(l2):
            parameters['gamma'].append(l2[i])
    l3 = st.multiselect('Degree of Polynomial',(2,3,4,5))
    if l3:
        for i,v in enumerate(l3):
            parameters['degree'].append(l3[i])
    l4 = st.multiselect('Kernel',('rbf','poly','sigmoid'))
    if l4:
        for i,v in enumerate(l4):
            parameters['kernel'].append(l4[i])
    submit_button = st.form_submit_button(label='Submit')
    if submit_button:
        st.write(f'Hyperparameters entered {parameters}')
        

In [ ]:
 parameters={'C':[],'gamma':[],'degree':[],'kernel':[]}
    
    numbers=st.text_input('Enter the hyperparameters for C in float format seprated by comma')
    #if numbers:
    l1=[float(i) for i in re.split('[^0-9.]', numbers) if i != ""]
    for i,v in enumerate(l1):
        parameters['C'].append(l1[i])
    op=st.text_input('Enter the hyperparameters for Gamma in float format seprated by comma')
    if op:
        
    l3 = st.multiselect('Degree of Polynomial',(2,3,4,5))
    
    l4 = st.multiselect('Kernel',('rbf','poly','sigmoid'))
    

In [ ]:
import streamlit as st
import pandas as pd
st.title('Lets explore session state and call back functions')
if 'nor' not in st.session_state:
    st.session_state['nor']=5
    

In [2]:
a={}
a['df']=0

In [3]:
a['df']+=1

In [4]:
a

{'df': 1}

In [5]:
a['df']+=2

In [6]:
submit_button = st.form_submit_button(label='Submit')

{'df': 3}

In [ ]:
def KMeans():
    

In [ ]:
def DBScanClustering():
    df=pd.read_csv(path)
    st.write(df)